In [2]:
pip install pandas python-dotenv keyring  email-validator

   ---------------------------------------- 0.0/331.1 kB ? eta -:--:--
   ------- -------------------------------- 61.4/331.1 kB 1.6 MB/s eta 0:00:01
   ----------------- ---------------------- 143.4/331.1 kB 1.7 MB/s eta 0:00:01
   --------------------- ------------------ 174.1/331.1 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 194.6/331.1 kB 1.1 MB/s eta 0:00:01
   ---------------------------------- ----- 286.7/331.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  327.7/331.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 331.1/331.1 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
df = pd.read_csv("banking.csv")


In [8]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv
import os

# ---------------------------------------------
# Step 1: Load environment variables (optional)
# ---------------------------------------------
# If you have a .env file with credentials
# Example:
# EMAIL_USER=your_email@gmail.com
# EMAIL_PASS=your_app_password
# RECEIVER_EMAIL=receiver@gmail.com
load_dotenv()

sender_email = os.getenv("EMAIL_USER", "rutuanne2002@gmail.com")
password = os.getenv("EMAIL_PASS", "clfcvgipbmyeghja")
receiver_email = os.getenv("RECEIVER_EMAIL", "rutujaanne12@gmail.com")

# ---------------------------------------------
# Step 2: Load your bank dataset
# ---------------------------------------------
df = pd.read_csv("banking.csv")  # 👈 replace with your dataset filename

# ---------------------------------------------
# Step 3: Define Alert Conditions
# ---------------------------------------------
alerts = []

# Example 1 — Loan default rate alert
if "Defaulted" in df.columns:
    default_rate = df["Defaulted"].mean() * 100
    if default_rate > 10:
        alerts.append(f"🚨 Default rate is {default_rate:.2f}% (above 10%)")

# Example 2 — High-value transaction alert
if "TransactionAmount" in df.columns:
    high_txn = df[df["TransactionAmount"] > 100000]
    if not high_txn.empty:
        alerts.append(f"💰 {len(high_txn)} transactions > ₹1,00,000 detected")

# Example 3 — Customer churn (if you have date columns)
if "CustomerID" in df.columns and "Month" in df.columns:
    month_summary = df.groupby("Month")["CustomerID"].nunique()
    if len(month_summary) >= 2:
        diff = month_summary.iloc[-1] - month_summary.iloc[-2]
        if diff < 0:
            alerts.append(f"📉 Customer count dropped by {-diff} from last month")

# If no alerts triggered
if not alerts:
    alerts.append("✅ All metrics are stable — no issues detected.")

# ---------------------------------------------
# Step 4: Format the Email
# ---------------------------------------------
msg = MIMEMultipart("alternative")
msg["Subject"] = "📊 Bank Data Alert Summary"
msg["From"] = sender_email
msg["To"] = receiver_email

html_content = f"""
<html>
<body>
    <h2>Daily Bank Data Alerts</h2>
    <ul>
        {''.join(f'<li>{a}</li>' for a in alerts)}
    </ul>
</body>
</html>
"""
msg.attach(MIMEText(html_content, "html"))

# ---------------------------------------------
# Step 5: Send Email
# ---------------------------------------------
try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
    print("✅ Email alert sent successfully!")
except Exception as e:
    print("❌ Error sending email:", e)


✅ Email alert sent successfully!
